Previous part

In [ ]:
from bs4 import BeautifulSoup
import requests
import html.parser
import pandas as pd
import numpy as np
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import csv                  
import webbrowser
import io
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
cols=['PostalCode','Borough','Neighborhood']
session=requests.Session()
r = session.get(url)
soup=BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', class_='wikitable')
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)
output_rows=output_rows[1:]
def stripping(lst):
    return [value.rstrip() for value in lst]
result=list(map(stripping,output_rows))
newresult=list(filter(lambda x: x[1]!='Not assigned',result))
df=pd.DataFrame(newresult, columns=cols)
df.sort_values(by='PostalCode')

Table with coordinates

In [3]:
coords=pd.read_csv('http://cocl.us/Geospatial_data')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Checking if dataframe['PostalCode'] from 1st part equals PostalCodes from coords 

In [4]:
qwe=df.sort_values(by='PostalCode')['PostalCode']
# ==coords['Postal Code']

qwe.head()
newqwe=qwe.reset_index(drop=True)

qwe1=coords['Postal Code']
print(all(newqwe==qwe1))

True


Since previous cell returned True, we can easily add two columns Latitude and Longitude

In [5]:
df=df.sort_values(by='PostalCode')

newdf=df.reset_index(drop=True)

newdf['Latitude']=coords['Latitude']
newdf['Longitude']=coords['Longitude']

newdf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood, Latitude, Longitude]
Index: []


In [ ]:
from sklearn.cluster import KMeans
kclusters = 5

kmeans = KMeans(n_clusters=kclusters, random_state=13).fit(res_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels

merged=pd.DataFrame(index=res_df.index)
merged['Latitude']=res_df['Latitude']
merged['Longitude']=res_df['Longitude']
merged['Cluster Labels']=kmeans.labels_
# res_df.insert(res_df.shape[1], 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged.head() # check the last columns!
# results['Cluster Labels']=[int(value) for value in results['Cluster Labels']]


In [ ]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged.index, merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
res_df['Cluster Labels']=merged['Cluster Labels']

In [ ]:
res_df[res_df['Cluster Labels']==0]

In [ ]:
first=res_df[res_df['Cluster Labels']==0]
second=res_df[res_df['Cluster Labels']==1]
third=res_df[res_df['Cluster Labels']==2]
fourth=res_df[res_df['Cluster Labels']==3]
fifth=res_df[res_df['Cluster Labels']==4]

In [ ]:
lst=[first,second,third,fourth,fifth]
for i in lst:
    i.drop('Latitude', axis=1, inplace=True)
    i.drop('Longitude', axis=1, inplace=True)
    i.drop('Cluster Labels', axis=1, inplace=True)
    print(i.head())


In [ ]:
analysis=[i.describe()[['count', 'mean','min',]] for i in lst]

In [ ]:
analysis